install and import packages

In [ ]:
# install BeautifulSoup
!conda install -c anaconda beautifulsoup4


In [7]:
!conda install -c conda-forge geopy --yes
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\jingy\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.0               |   py38haa244fe_1         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  conda                                4.9.2-py38haa244fe_0 --> 4.10.0-py38haa244fe_1




conda-4.10.0         | 3.1 MB    |            |   0% 
conda-4.10.0         | 3.1 MB    |            |   1% 
conda-4.10.0         | 3.1 MB    | 2          |   3% 
conda-4.10.0         | 3.1 MB    | 4          |   5% 
conda-4.10.0         | 3.1 MB    | 6          |   7% 
conda-4.10.0         | 3.1 MB    | 8          |   9% 
conda-4.10.0         | 3.1 MB    | #      

In [ ]:
!pip install geocoder

In [1]:
#import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import geocoder 
import folium

### step 1. create toronto neighborhood dataframe - df

In [2]:
# requests data from wiki ppage and create the dataframe
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url)
#print(r)
soup=BeautifulSoup(r.text,'html.parser')
#print(soup)

table=soup.find_all('table')[0]
#print(table)
all_postalcode=table.find_all('p')
#print(len(all_postalcode))
data=[]

PostalCode=[]
Borough=[]
Neighborhood=[]
for p in all_postalcode:
    
    PCode=p.find('b').text
    #print(PCode)
    Borough_i=p.text.replace(PCode,'')
    #print(Borough_i)
    if Borough_i=="Not assigned\n":
        pass
    else:
        PostalCode=PCode
        Borough_Nei=Borough_i.split('(')
        #print(Borough_Nei)
        Borough=Borough_Nei[0]
        #print(Borough)
        Neighborhood=Borough_Nei[1].replace(')\n','')
        Neighborhood=Neighborhood.replace('\n','')
        Neighborhood=Neighborhood.split(')')[0]
        #print(PostalCode, Borough, Neighborhood)      
        data.append([PostalCode,Borough,Neighborhood])
#print(data)
df=pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood'])
print(df[df['PostalCode']=='M3N'])
df=df.drop_duplicates()
print(df)


   PostalCode     Borough Neighborhood
60        M3N  North York    Downsview
    PostalCode                                            Borough  \
0          M3A                                         North York   
1          M4A                                         North York   
2          M5A                                   Downtown Toronto   
3          M6A                                         North York   
4          M7A                                       Queen's Park   
..         ...                                                ...   
98         M8X                                          Etobicoke   
99         M4Y                                   Downtown Toronto   
100        M7Y  East TorontoBusiness reply mail Processing Cen...   
101        M8Y                                          Etobicoke   
102        M8Z                                          Etobicoke   

                                          Neighborhood  
0                                   

### step 2. get the latitude and longitude of all the boroughs and create new data frame df_new

In [8]:
# method 1 - get the latitude and longitude by geocoder
borough_list=df['Borough'].values

Latitude=[]
Longitude=[]


for borough_each in borough_list:
    #print(borough_each)
    try:
        geolocator = Nominatim(user_agent="toronto_explorer")

        g=geolocator.geocode('{}, Toronto, Ontario'.format(borough_each))
        #print("g is {}".format(g) )
      
                      
        Latitude.append(g.latitude)
        Longitude.append(g.longitude)
    except:
        Latitude.append(0)
        Longitude.append(0)
    
        
#print(Latitude,Longitude)
df_complete=df.copy()

df_complete['Latitude']=Latitude
df_complete['Longitude']=Longitude
print(df_complete)
        
      
        
# method 2 - get the latitude abd longitude by csv_file        

'''df_lat_lng=pd.read_csv('https://cocl.us/Geospatial_data')
#print(df_lat_lng)
    
df_new=pd.merge(df,df_lat_lng, left_on='PostalCode',right_on='Postal Code').drop(columns='Postal Code')
print(df_new)'''




    PostalCode                                            Borough  \
0          M3A                                         North York   
1          M4A                                         North York   
2          M5A                                   Downtown Toronto   
3          M6A                                         North York   
4          M7A                                       Queen's Park   
..         ...                                                ...   
98         M8X                                          Etobicoke   
99         M4Y                                   Downtown Toronto   
100        M7Y  East TorontoBusiness reply mail Processing Cen...   
101        M8Y                                          Etobicoke   
102        M8Z                                          Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.754326 -79.449117  
1            

"df_lat_lng=pd.read_csv('https://cocl.us/Geospatial_data')\n#print(df_lat_lng)\n    \ndf_new=pd.merge(df,df_lat_lng, left_on='PostalCode',right_on='Postal Code').drop(columns='Postal Code')\nprint(df_new)"

In [9]:
df_complete=df_complete[df_complete['Latitude']!=0]
print(df_complete)

    PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A      Queen's Park   
..         ...               ...   
97         M5X  Downtown Toronto   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.754326 -79.449117  
1                                     Victoria Village  43.754326 -79.449117  
2                           Regent Park / Harbourfront  43.656322 -79.380916  
3                    Lawrence Manor / Lawrence Heights  43.754326 -79.449117  
4                        Ontario Provincial Government  43.659659 -79.390340  
..                                                 ...        ...        ...  
97            

In [10]:
# make neighborhood column into list
df_new_1=df_complete.copy()
df_new_1['Neighborhood']=df_new_1['Neighborhood'].str.split('/')
print(df_new_1)

    PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A      Queen's Park   
..         ...               ...   
97         M5X  Downtown Toronto   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                          [Parkwoods]  43.754326 -79.449117  
1                                   [Victoria Village]  43.754326 -79.449117  
2                        [Regent Park ,  Harbourfront]  43.656322 -79.380916  
3                 [Lawrence Manor ,  Lawrence Heights]  43.754326 -79.449117  
4                      [Ontario Provincial Government]  43.659659 -79.390340  
..                                                 ...        ...        ...  
97          [F

### step 3. Create a map of Toronto with neighborhoods superimposed on top.

3-1 get the complete dataframe (containing latitude and longitude of all neighborhoods)

In [11]:

# explode each neighborhood into a row
df_final=df_new_1.explode('Neighborhood')

# replace the wrong name in the dataframe
df_final['Neighborhood']=df_final['Neighborhood'].str.replace('Beaumond Heights ' , 'Beaumonde Heights')
print(df_final)

# get the latitude and longitude of each neighborhood
neighborhood_list=df_final['Neighborhood'].values
Latitude=[]
Longitude=[] 
for each_neighborhood in neighborhood_list:
    #print(each_neighborhood)
    try:
        geolocator = Nominatim(user_agent="toronto_explorer")
        g=geolocator.geocode('{}, Toronto, Ontario'.format(each_neighborhood))
        #print("g is {}".format(g) )
                     
        Latitude.append(g.latitude)
        Longitude.append(g.longitude)
    except:
        Latitude.append(0)
        Longitude.append(0)
        
#print(Latitude,Longitude)

df_final['Latitude']=Latitude
df_final['Longitude']=Longitude
print(df_final)

    PostalCode           Borough                Neighborhood   Latitude  \
0          M3A        North York                   Parkwoods  43.754326   
1          M4A        North York            Victoria Village  43.754326   
2          M5A  Downtown Toronto                Regent Park   43.656322   
2          M5A  Downtown Toronto                Harbourfront  43.656322   
3          M6A        North York             Lawrence Manor   43.754326   
..         ...               ...                         ...        ...   
102        M8Z         Etobicoke                  Mimico NW   43.643556   
102        M8Z         Etobicoke         The Queensway West   43.643556   
102        M8Z         Etobicoke             South of Bloor   43.643556   
102        M8Z         Etobicoke   Kingsway Park South West   43.643556   
102        M8Z         Etobicoke       Royal York South West  43.643556   

     Longitude  
0   -79.449117  
1   -79.449117  
2   -79.380916  
2   -79.380916  
3   -79.449117

In [12]:
df_final=df_final[df_final['Latitude']!=0]
print(df_final)

    PostalCode           Borough                Neighborhood   Latitude  \
0          M3A        North York                   Parkwoods  43.758800   
1          M4A        North York            Victoria Village  43.732658   
2          M5A  Downtown Toronto                Regent Park   43.660706   
2          M5A  Downtown Toronto                Harbourfront  43.640080   
3          M6A        North York             Lawrence Manor   43.722079   
..         ...               ...                         ...        ...   
102        M8Z         Etobicoke                  Mimico NW   43.616677   
102        M8Z         Etobicoke         The Queensway West   43.623618   
102        M8Z         Etobicoke             South of Bloor   43.667662   
102        M8Z         Etobicoke   Kingsway Park South West   43.647381   
102        M8Z         Etobicoke       Royal York South West  43.648183   

     Longitude  
0   -79.320197  
1   -79.311189  
2   -79.360457  
2   -79.380150  
3   -79.437507

3-2 create the map

Use geopy library to get the latitude and longitude values of Toronto.

In [13]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top.

In [14]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'OQMBZOQH4UD5GJCMBNF1T1AIYWIXWAT1BKFFMGBGKOBS1KJU' # your Foursquare ID
CLIENT_SECRET = 'YFMPLQAXMVGUER1HN2GJ0W23MOKM3Z5TKUOS4TQFJ2E2KZXD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OQMBZOQH4UD5GJCMBNF1T1AIYWIXWAT1BKFFMGBGKOBS1KJU
CLIENT_SECRET:YFMPLQAXMVGUER1HN2GJ0W23MOKM3Z5TKUOS4TQFJ2E2KZXD


select only boroughs that  contain the word "Toronto"

In [33]:
Toronto_boroughs=df_final[df_final['Borough'].str.contains('Toronto')]
Toronto_boroughs.reset_index(drop=True,inplace=True)
Toronto_boroughs.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
1,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
2,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
3,M4E,East Toronto,The Beaches,43.671024,-79.296712
4,M5E,Downtown Toronto,Berczy Park,43.647984,-79.375396


try 1 neighborhood with foursquare api, get the names and categories of nearby venues

In [118]:
neighborhood_1=Toronto_boroughs.loc[0,'Neighborhood']
latitude_1=Toronto_boroughs.loc[0,'Latitude']
longitude_1=Toronto_boroughs.loc[0,'Longitude']
print('Latitude and longitude of {} is {},{}'.format(neighborhood_1,latitude_1,longitude_1,))

limit=100
radius=100
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
     CLIENT_ID,
     CLIENT_SECRET,
     VERSION,
     latitude_1,
     longitude_1,
     radius,
     limit)
print(url)

import requests
from pandas.io.json import json_normalize
results=requests.get(url).json()
#print(len(results))
#print(results['response']['groups'][0])
venues=results['response']['groups'][0]['items']
#print(type(venues))
#print(len(venues))
#print(venues[0])
nearby_venues=json_normalize(venues)
#print(nearby_venues)
#print(type(nearby_venues))
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]
print(nearby_venues)

Latitude and longitude of Regent Park  is 43.6607056,-79.3604569
https://api.foursquare.com/v2/venues/explore?&client_id=OQMBZOQH4UD5GJCMBNF1T1AIYWIXWAT1BKFFMGBGKOBS1KJU&client_secret=YFMPLQAXMVGUER1HN2GJ0W23MOKM3Z5TKUOS4TQFJ2E2KZXD&v=20180605&ll=43.6607056,-79.3604569&radius=100&limit=100
                   venue.name  \
0  Regent Park Aquatic Centre   
1          Shoppers Drug Mart   
2                     Wendy’s   

                                    venue.categories  venue.location.lat  \
0  [{'id': '4bf58dd8d48988d15e941735', 'name': 'P...           43.660600   
1  [{'id': '4bf58dd8d48988d10f951735', 'name': 'P...           43.660204   
2  [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...           43.660403   

   venue.location.lng  
0          -79.361392  
1          -79.361332  
2          -79.360629  


<ipython-input-118-d77fdc290977>:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues=json_normalize(venues)


define a function to extract the category of venues

In [119]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
       
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    
# filter the category of each row
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)
print(nearby_venues)

# clean the columns
nearby_venues.columns=[col.split('.')[-1] for col in nearby_venues.columns]
nearby_venues
    

                   venue.name      venue.categories  venue.location.lat  \
0  Regent Park Aquatic Centre                  Pool           43.660600   
1          Shoppers Drug Mart              Pharmacy           43.660204   
2                     Wendy’s  Fast Food Restaurant           43.660403   

   venue.location.lng  
0          -79.361392  
1          -79.361332  
2          -79.360629  


,name,categories,lat,lng
0,Regent Park Aquatic Centre,Pool,43.660600,-79.361392
1,Shoppers Drug Mart,Pharmacy,43.660204,-79.361332
2,Wendy’s,Fast Food Restaurant,43.660403,-79.360629


### explore Toronto neighborhoods

In [134]:
def explore_nearby_venues(names,latitudes,longitudes,radius=500):
    venue_lists=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            limit)
        
        results=requests.get(url).json()
        venues=results['response']['groups'][0]['items']
        
        for venues_1 in venues:
            venue=venues_1['venue']['name']
            category=venues_1['venue']['categories']
            latitude=venues_1['venue']['location']['lat']
            longitude=venues_1['venue']['location']['lng']
            venue_lists.append([name,lat,lng,venue,category,latitude,longitude])
        
    nearby_venues=pd.DataFrame(venue_lists, columns=['neighborhood','n_lat','n_lng','venue','category','venue_lat','venue_lng'])
    return(nearby_venues)

In [146]:

toronto_venues=explore_nearby_venues(Toronto_boroughs['Neighborhood'],Toronto_boroughs['Latitude'],Toronto_boroughs['Longitude'])

toronto_venues.head()

,neighborhood,n_lat,n_lng,venue,category,venue_lat,venue_lng
0,Regent Park,43.660706,-79.360457,Regent Park Aquatic Centre,"[{'id': '4bf58dd8d48988d15e941735', 'name': 'P...",43.660600,-79.361392
1,Regent Park,43.660706,-79.360457,Sumach Espresso,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.658135,-79.359515
2,Regent Park,43.660706,-79.360457,Daniels Spectrum,"[{'id': '4bf58dd8d48988d1f2931735', 'name': 'P...",43.660137,-79.361808
3,Regent Park,43.660706,-79.360457,Thai To Go,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",43.663418,-79.360710
4,Regent Park,43.660706,-79.360457,Sukhothai,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",43.658444,-79.365681


In [149]:
print(toronto_venues.shape)
def get_category_type_1(row):
    try:
        categories_list=row['category']
    except:
        categories_list=row['venue.categories']
       
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
toronto_venues['category']=toronto_venues.apply(get_category_type_1,axis=1)
toronto_venues.head()

(3310, 7)


,neighborhood,n_lat,n_lng,venue,category,venue_lat,venue_lng
0,Regent Park,43.660706,-79.360457,Regent Park Aquatic Centre,Pool,43.660600,-79.361392
1,Regent Park,43.660706,-79.360457,Sumach Espresso,Coffee Shop,43.658135,-79.359515
2,Regent Park,43.660706,-79.360457,Daniels Spectrum,Performing Arts Venue,43.660137,-79.361808
3,Regent Park,43.660706,-79.360457,Thai To Go,Thai Restaurant,43.663418,-79.360710
4,Regent Park,43.660706,-79.360457,Sukhothai,Thai Restaurant,43.658444,-79.365681


In [150]:
toronto_venues.groupby('neighborhood').count()

,n_lat,n_lng,venue,category,venue_lat,venue_lng
neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,26,26,26,26,26,26
Cabbagetown,48,48,48,48,48,48
Chinatown,62,62,62,62,62,62
Deer Park,56,56,56,56,56,56
...,...,...,...,...,...,...
The Annex,39,39,39,39,39,39
The Beaches,45,45,45,45,45,45
The Danforth West,27,27,27,27,27,27


In [156]:

unique_l=toronto_venues['category'].unique()
#print(unique_l)
print('there are {} unique categories.'.format(len(toronto_venues['category'].unique())))

there are 288 unique categories.


#### analyze each neighborhood

In [161]:
toronto_onehot=pd.get_dummies(toronto_venues[['category']],prefix="",prefix_sep="")
toronto_onehot['Neighborhood']=toronto_venues['neighborhood']
fixed_columns=[toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[fixed_columns]
#print(toronto_onehot.head())
print(toronto_onehot.shape)

(3310, 288)


In [165]:
toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.030000,0.0,0.0,...,0.0,0.0,0.010000,0.000000,0.0,0.000000,0.0,0.010000,0.0,0.0
1,Bathurst Quay,0.000000,0.0,0.0,0.038462,0.038462,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
2,Cabbagetown,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
3,Chinatown,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.032258,0.016129,0.0,0.032258,0.0,0.016129,0.0,0.0
4,Deer Park,0.017857,0.0,0.0,0.000000,0.000000,0.0,0.017857,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.017857,0.0,0.000000,0.0,0.0


In [167]:
toronto_grouped.shape

(69, 288)

### Cluster Neighborhoods

In [211]:
kclusters=5
toronto_grouped_cluster=toronto_grouped.drop(columns=['Neighborhood'])
toronto_grouped_cluster.head()

from sklearn.cluster import KMeans

kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_cluster)
kmeans.labels_[:10]
set(kmeans.labels_)


{0, 1, 2, 3, 4}

In [192]:
num_top_venues = 5
#print(toronto_grouped.head())

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide ----
                 venue  freq
0          Coffee Shop  0.07
1           Restaurant  0.04
2       Cosmetics Shop  0.04
3                 Café  0.04
4  American Restaurant  0.03


---- Bathurst Quay ----
             venue  freq
0      Coffee Shop  0.15
1             Café  0.12
2    Grocery Store  0.08
3  Harbor / Marina  0.08
4             Park  0.08


---- Cabbagetown----
               venue  freq
0         Restaurant  0.08
1               Café  0.08
2        Coffee Shop  0.06
3  Indian Restaurant  0.04
4             Bakery  0.04


---- Chinatown ----
                           venue  freq
0                           Café  0.06
1                    Coffee Shop  0.05
2             Mexican Restaurant  0.05
3  Vegetarian / Vegan Restaurant  0.03
4            Arts & Crafts Store  0.03


---- Deer Park----
                venue  freq
0         Coffee Shop  0.11
1       Grocery Store  0.07
2    Sushi Restaurant  0.07
3  Italian Restaurant  0.07
4     Thai Restaurant  0.05



In [193]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [195]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Cosmetics Shop,Restaurant,Café,Italian Restaurant,Japanese Restaurant,Gastropub,Seafood Restaurant,American Restaurant,Gym
1,Bathurst Quay,Coffee Shop,Café,Grocery Store,Park,Harbor / Marina,Bank,Japanese Restaurant,Diner,Gym,Sushi Restaurant
2,Cabbagetown,Café,Restaurant,Coffee Shop,Pizza Place,Beer Store,Pub,Italian Restaurant,Bakery,Gastropub,Diner
3,Chinatown,Café,Mexican Restaurant,Coffee Shop,Ramen Restaurant,Clothing Store,Park,Arts & Crafts Store,Bar,Dessert Shop,Vegetarian / Vegan Restaurant
4,Deer Park,Coffee Shop,Grocery Store,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Bank,Bagel Shop,Pizza Place,Café,Gym


In [197]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_boroughs

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!'''

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,0,Coffee Shop,Restaurant,Thai Restaurant,Fast Food Restaurant,Grocery Store,Beer Store,Pub,Auto Dealership,Intersection,Electronics Store
1,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Pizza Place,Fried Chicken Joint,Park,Chinese Restaurant,Plaza
2,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704,0,Coffee Shop,Pizza Place,Café,Grocery Store,Pharmacy,Restaurant,Bistro,Bar,Bank,Bakery
3,M4E,East Toronto,The Beaches,43.671024,-79.296712,0,Beach,Breakfast Spot,Japanese Restaurant,Bar,Pizza Place,Nail Salon,Park,Liquor Store,Juice Bar,Jewelry Store
4,M5E,Downtown Toronto,Berczy Park,43.647984,-79.375396,0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Cocktail Bar,Gym


In [198]:
import numpy as np
map_clusters=folium.Map(location=[latitude, longitude],zoom_start=11)
map_clusters
x=np.arange(kclusters)
print(range(kclusters))
ys=[i+x+(i*x)**2 for i in range(kclusters)]
ys

import matplotlib.cm as cm
import matplotlib.colors as colors
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
colors_array
rainbow=[colors.rgb2hex(i) for i in colors_array]
rainbow

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



range(0, 5)


In [202]:
cluster0=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster2=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster3=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster4=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [200]:
cluster0

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Restaurant,Thai Restaurant,Fast Food Restaurant,Grocery Store,Beer Store,Pub,Auto Dealership,Intersection,Electronics Store
1,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Pizza Place,Fried Chicken Joint,Park,Chinese Restaurant,Plaza
2,Downtown Toronto,0,Coffee Shop,Pizza Place,Café,Grocery Store,Pharmacy,Restaurant,Bistro,Bar,Bank,Bakery
3,East Toronto,0,Beach,Breakfast Spot,Japanese Restaurant,Bar,Pizza Place,Nail Salon,Park,Liquor Store,Juice Bar,Jewelry Store
4,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Cocktail Bar,Gym
...,...,...,...,...,...,...,...,...,...,...,...,...
64,Downtown Toronto,0,Coffee Shop,Pizza Place,Café,Grocery Store,Pharmacy,Restaurant,Bistro,Bar,Bank,Bakery
65,Downtown Toronto,0,Café,Restaurant,Coffee Shop,Pizza Place,Beer Store,Pub,Italian Restaurant,Bakery,Gastropub,Diner
66,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Gym,Japanese Restaurant,Steakhouse,Seafood Restaurant,Deli / Bodega,American Restaurant
67,Downtown Toronto,0,Breakfast Spot,Sporting Goods Shop,Bank,Chinese Restaurant,Gas Station,Outdoor Supply Store,Fast Food Restaurant,Sandwich Place,Bubble Tea Shop,Persian Restaurant


In [206]:
cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,1,Park,Beer Store,Women's Store,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market


In [207]:
cluster2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Central Toronto,2,Spa,Trail,Skating Rink,Event Space,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant


In [208]:
cluster3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Downtown Toronto,3,Airport,Airport Terminal,Women's Store,Farmers Market,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant


In [209]:
cluster4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Central Toronto,4,Playground,Park,Bank,Home Service,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
63,Downtown Toronto,4,Park,Playground,Bike Trail,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
